In [ ]:
from __future__ import absolute_import, division, print_function
import json
import time

import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from metric_utils import *
import torchmetrics

## Predictions Format

The model accepts predictions as JSON files in the following format
(The notation in caps with a dollar sign (`$IMG_NAME`) means that such key will be replaced by the actual value of the property)
```json
    {
        "$IMG_NAME": {
            "labels": [$CLASS_IDX: int, ..., $CLASS_IDX: int],
            "boxes": [[$X1, $X2, $Y1, $Y2], ..., [$X1, $X2, $Y1, $Y2]],
            "scores": [$CONFIDENCE_SCORE: float, ..., $CONFIDENCE_SCORE: float]
        },

        ...
        
        "$IMG_NAME": {
            "labels": [$CLASS_IDX: int, ..., $CLASS_IDX: int],
            "boxes": [[$X1, $X2, $Y1, $Y2], ..., [$X1, $X2, $Y1, $Y2]],
            "scores": [$CONFIDENCE_SCORE: float, ..., $CONFIDENCE_SCORE: float]
        }
    }
```

## Ground Truth Format

Compliant to [COCO format]()
```json
{
    "image": {
        "id": int, 
        "width": int, 
        "height": int, 
        "file_name": str
    },
    "annotations": {
        "id": int, 
        "image_id": int, 
        "category_id": int, 
        "segmentation": RLE or [polygon], 
        "area": float, 
        "bbox": [x,y,width,height], 
        "iscrowd": 0 or 1,
    },

    "categories": [{
        "id": int, 
        "name": str
    }]
}
```

## Visualization setup

In [ ]:
sns.set_style('white')
sns.set_context('poster')

COLORS = [
    '#1f77b4', '#aec7e8', '#ff7f0e', '#ffbb78', '#2ca02c',
    '#98df8a', '#d62728', '#ff9896', '#9467bd', '#c5b0d5',
    '#8c564b', '#c49c94', '#e377c2', '#f7b6d2', '#7f7f7f',
    '#c7c7c7', '#bcbd22', '#dbdb8d', '#17becf', '#9edae5']

## Loading gt and pred annotations

In [ ]:
with open('test.json') as infile:
    annotations = json.load(infile)

with open('rg-boxes copy.json') as infile:
    pred_boxes = json.load(infile)
    
id_to_filename = {}
for img in annotations['images']:
    id_to_filename[img['id']] = img['file_name'].split('.')[0]

class_id_to_name = {}
for cl in annotations['categories']:
    class_id_to_name[cl['id']] = cl['name']

gt_boxes = {}
for ann in annotations['annotations']:
    img_id = ann['image_id']
    img_name = id_to_filename[img_id]
    if img_name not in gt_boxes:
        gt_boxes[img_name] = {'boxes': [], 'labels': []}
    bbox = ann['bbox']
    bbox[2] += bbox[0]
    bbox[3] += bbox[1]
    class_id = ann['category_id']
    gt_boxes[img_name]['boxes'].append(bbox)
    gt_boxes[img_name]['labels'].append(class_id)

## Fix missing keys

In [ ]:
for miss_k in (set(pred_boxes) - set(gt_boxes)):
    del pred_boxes[miss_k]

In [ ]:
set(pred_boxes) - set(gt_boxes)

In [ ]:
import torch
pred_list = []
for img in sorted(pred_boxes):
    for k, v in pred_boxes[img].items():
        pred_boxes[img][k] = torch.tensor(v)
    pred_list.append(pred_boxes[img])

gt_list = []
for img in sorted(gt_boxes):
    for k, v in gt_boxes[img].items():
        gt_boxes[img][k] = torch.tensor(v)
    gt_list.append(gt_boxes[img])

In [ ]:
map50 = torchmetrics.MAP(class_metrics=True)
map50.update(pred_list, gt_list)
map50.compute()

In [ ]:
# Split all groundtruths and detections into subsets by the object detected
gt_boxes_source, pred_boxes_source = split_boxes_by_class(gt_boxes, pred_boxes, 'source')
gt_boxes_sidelobe, pred_boxes_sidelobe = split_boxes_by_class(gt_boxes, pred_boxes, 'sidelobe')
gt_boxes_galaxy, pred_boxes_galaxy = split_boxes_by_class(gt_boxes, pred_boxes, 'galaxy')

## mAP@0.5

In [ ]:
# Run per class mAP calculation for each class
iou_thr = 0.5
# calculate source mAP
data = get_avg_precision_at_iou(gt_boxes_source, pred_boxes_source, iou_thr=iou_thr)
print('Source avg precision: {:.4f}'.format(data['avg_prec']))
# calculate sidelobe mAP
data = get_avg_precision_at_iou(gt_boxes_sidelobe, pred_boxes_sidelobe, iou_thr=iou_thr)
print('Sidelobe avg precision: {:.4f}'.format(data['avg_prec']))
# calculate galaxy mAP
data = get_avg_precision_at_iou(gt_boxes_galaxy, pred_boxes_galaxy, iou_thr=iou_thr)
print('Galaxy avg precision: {:.4f}'.format(data['avg_prec']))

# Runs it for one IoU threshold for all predictions
start_time = time.time()
data = get_avg_precision_at_iou(gt_boxes, pred_boxes, iou_thr=iou_thr)
end_time = time.time()
print('Single IoU calculation took {:.4f} secs'.format(end_time - start_time))
print('avg precision: {:.4f}'.format(data['avg_prec']))

## mAP@0.5:0.95 + PR Curve

In [ ]:
start_time = time.time()
ax = None
avg_precs = []
iou_thrs = []
for idx, iou_thr in enumerate(np.linspace(0.5, 0.95, 10)):
    data = get_avg_precision_at_iou(gt_boxes, pred_boxes, iou_thr=iou_thr)
    avg_precs.append(data['avg_prec'])
    iou_thrs.append(iou_thr)

    precisions = data['precisions']
    recalls = data['recalls']
    ax = plot_pr_curve(
        precisions, recalls, label='{:.2f}'.format(iou_thr), color=COLORS[idx*2], ax=ax)

# prettify for printing:
avg_precs = [float('{:.4f}'.format(ap)) for ap in avg_precs]
iou_thrs = [float('{:.4f}'.format(thr)) for thr in iou_thrs]
print('map: {:.2f}'.format(100*np.mean(avg_precs)))
print('avg precs: ', avg_precs)
print('iou_thrs:  ', iou_thrs)
plt.legend(loc='upper right', title='IOU Thr', frameon=True)
for xval in np.linspace(0.0, 1.0, 11):
    plt.vlines(xval, 0.0, 1.1, color='gray', alpha=0.3, linestyles='dashed')
end_time = time.time()
print('\nPlotting and calculating mAP takes {:.4f} secs'.format(end_time - start_time))
plt.show()

## Plot PR Curve per class

In [ ]:
# Super-Imposed per Class
start_time = time.time()
ax = None
avg_precs = []
iou_thrs = []
classes = ['source', 'sidelobe', 'galaxy']
for i, classification in enumerate(classes):
    for iou_thr in np.linspace(0.5, 0.6, 2):
        if classification == 'source':
            data = get_avg_precision_at_iou(gt_boxes_source, pred_boxes_source, iou_thr=iou_thr)
            colour = 'blue'
        elif classification == 'sidelobe':
            data = get_avg_precision_at_iou(gt_boxes_sidelobe, pred_boxes_sidelobe, iou_thr=iou_thr)
            colour = 'red'
        elif classification == 'galaxy':
            data = get_avg_precision_at_iou(gt_boxes_galaxy, pred_boxes_galaxy, iou_thr=iou_thr)
            colour = 'green'
        avg_precs.append(data['avg_prec'])
        iou_thrs.append(iou_thr)

        precisions = data['precisions']
        recalls = data['recalls']

        if iou_thr == 0.5:
            line_style = 'solid'
        elif iou_thr == 0.6:
            line_style = 'dashed'
        # ax = plot_pr_curve(precisions, recalls, label=classification+' @ {:.2f}'.format(iou_thr),
        #                    color=COLORS[i*len(classes) + j], ax=ax)
        ax = plot_pr_curve(precisions, recalls, label=classification + ' @ {:.2f}'.format(iou_thr),
                            color=colour, ax=ax, linestyle=line_style)


# prettify for printing:
avg_precs = [float('{:.4f}'.format(ap)) for ap in avg_precs]
iou_thrs = [float('{:.4f}'.format(thr)) for thr in iou_thrs]
print('map: {:.2f}'.format(100 * np.mean(avg_precs)))
print('avg precs: ', avg_precs)
print('iou_thrs:  ', iou_thrs)
plt.title('Precision-Recall Curves per Class')
plt.legend(loc='lower right', title='Class @ IOU Thr', frameon=True)
for xval in np.linspace(0.0, 1.0, 11):
    plt.vlines(xval, 0.0, 1.1, color='gray', alpha=0.3, linestyles='dashed')
end_time = time.time()
print('\nPlotting and calculating mAP takes {:.4f} secs'.format(end_time - start_time))
plt.show()